In [41]:
# initial setup
# %run "../../../common/0_notebooks_base_setup.py"


Los temas que vimos en esta parte son limpieza de datos, expresiones regulares, funciones lambda, apply .

La idea de esta práctica es ejercitar los puntos que vamos a necesitar en la clase presencial.

Usaremos el dataset de las obras del Met (The Metropolitan Museum of Art)

https://github.com/metmuseum/openaccess/



## Ejercicio 1
Vamos a leer en la variable `data` los datos del archivo /M2/CLASE_05_Limpieza_de_datos/Data/MetObjects_sample.csv en un `DataFrame` de pandas con el método `read_csv` 

Veamos de qué tipos de datos son las columnas.


In [42]:
import pandas as pd
data = pd.read_csv("../Data/MetObjects_sample.csv")
data.sample(3)

,Unnamed: 0,Object Number,Is Highlight,Is Public Domain,Is Timeline Work,Object ID,Department,AccessionYear,Object Name,Title,...,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL
3728,165237,48.116.1,False,False,False,269201,Photographs,1948,Photograph,The Feast of Solitude,...,NaN,NaN,Photographs,© Copyright The Historic New Orleans Collectio...,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Ruins,http://vocab.getty.edu/page/aat/300008057
4147,414757,61.663.226(b),False,False,False,712064,Drawings and Prints,1961,Print,"Picture Book, final page (colophon), combined ...",...,NaN,NaN,Prints,"© 2020 Artists Rights Society (ARS), New York",http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN
650,350289,O.C.401,False,False,False,555290,Egyptian Art,NaN,Amulet,Amulet,...,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN


## Ejercicio 2
¿Qué campos tienen valores nulos? ¿Qué porcentaje de nulos tienen cada uno de ellos?

In [43]:
null_values_data = data.isnull().sum()
total = len(data)
percentage_null_values = null_values_data.apply(lambda x: x/total * 100)
percentage_null_values.astype("float")

Unnamed: 0                   0.000000
Object Number                0.000000
Is Highlight                 0.000000
Is Public Domain             0.000000
Is Timeline Work             0.000000
Object ID                    0.000000
Department                   0.000000
AccessionYear               21.083702
Object Name                  1.855366
Title                       17.478389
Culture                     51.127978
Period                      72.190597
Dynasty                     96.120599
Reign                       98.503057
Portfolio                   96.963947
Artist Role                 49.736454
Artist Prefix               84.482395
Artist Display Name         49.441282
Artist Display Bio          55.471221
Artist Suffix               98.334388
Artist Alpha Sort           49.441282
Artist Nationality          66.814253
Artist Begin Date           55.787476
Artist End Date             55.977230
Artist Gender               79.084967
Artist ULAN URL             62.492094
Artist Wikid

## Ejercicio 3

Analicemos la columna Object Date

¿Qué formato tienen los valores de este campo? ¿Qué patrones pueden identificar?

Sugerencia: recuerden el método value_counts
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html

In [44]:
object_data_series = data["Object Date"]
object_data_series.sample(40)

527                           1912
1533                           NaN
1290                           NaN
816              ca. 1070–343 B.C.
936                            NaN
3271                  17th century
3513                          1848
263                           1561
1761                           NaN
373                           1910
2360                           NaN
2132                           NaN
663                            NaN
3062                          1970
984                           1935
557                           1902
1212                           NaN
3286           fall/winter 1987–88
166              late 19th century
191                           1909
1288                           NaN
2156                           NaN
3577                 July 15, 1812
3602      ca. 5th–7th century A.D.
3                             1909
4294              ca. 883–859 B.C.
3413            ca. 1390–1353 B.C.
1403                           NaN
1378                

## Ejercicio 4

Usando expresiones regulares, apply y funciones lambda, creemos un nuevo campo "ObjectDateClean" que sea de tipo numérico y tenga el valor del año corresponiente a cada registro extrayéndolo del campo ObjectDate.

¿Qué características tienen los valores de los registros que no verificaron ningún patrón de los definidos?

Ayuda: Identifiquemos los patrones y decidamos cómo extraer el año de los valores de los registros que verifican cada uno de esos patrones. Después, de a un patrón, usemos la estrategia que definimos antes para extraer el valor del año de los registros que lo verifican.

In [45]:
import re 
import numpy as np
pattern_years = "\d{4}"
regex = re.compile(pattern_years)
data_year_match = object_data_series.apply(lambda x:  x if x is np.NaN else regex.search(x))
mask_data_year_match = data_year_match.notnull()
mask_data_year_match


0       False
1        True
2        True
3        True
4        True
        ...  
4738     True
4739     True
4740    False
4741     True
4742     True
Name: Object Date, Length: 4743, dtype: bool

In [46]:
data.loc[mask_data_year_match, "ObjectDateClean"] = data_year_match[mask_data_year_match].apply(lambda x: x.group(0))
data.loc[mask_data_year_match, ["Object Date", "ObjectDateClean"]]

,Object Date,ObjectDateClean
1,1642,1642
2,1911,1911
3,1909,1909
4,"June 20, 1867",1867
5,ca. 1878,1878
...,...,...
4736,1825–35,1825
4738,"1782, 2nd edition",1782
4739,ca. 1938,1938
4741,1669,1669


Posiblemente veamos que nos faltó considerar:
* los valores terminados en BC o B.C., y marcar esos años como negativos 
* considerar los períodos que tienen siglos, en lugar de años
* borrar las C. que aparecen de sufijo
* Considerar pallabras mals escritas como "cenutry"

Si tienen ganas de seguir practicando pueden resolver algunos de éstos.


In [52]:
century_pattern = "(?P<century>\d\d)((|th) (C|c)entury)"
century_pattern_regex = re.compile(century_pattern)

data_century_match = object_data_series.apply(lambda x: x if x is np.NaN else century_pattern_regex.search(x))

mask_century_period_match_notnull = data_century_match.notnull()

data_century = data_century_match[mask_century_period_match_notnull].apply(lambda x: x.group("century"))

data.loc[mask_century_period_match_notnull, "ObjectDateClean"] = \
    data_century_match[mask_century_period_match_notnull].apply(lambda x: x.group("century"))


In [53]:
data.loc[mask_century_period_match_notnull, ["Object Date", "ObjectDateClean"]]

,Object Date,ObjectDateClean
0,20th century,20
9,11th–13th century,13
26,19th century,19
27,17th century,17
29,19th century,19
...,...,...
4708,late 19th century,19
4710,18th century,18
4728,first half 18th century,18
4730,early 13th century,13
